# Анализ сервиса для чтения книг (SQL)

Цель исследования: изучить базу данных крупного сервиса для чтения книг по подписке с целью формулирования ценностного предложения для нового продукта.


## Подключение библиотек и вывод первых строк

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import text, create_engine
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

In [2]:
def print(query):
    display(pd.io.sql.read_sql(query, con = engine))

### вывод строк таблицы books

In [3]:
books = ''' SELECT * FROM books'''
print(books)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268
...,...,...,...,...,...,...
995,996,571,Wyrd Sisters (Discworld #6; Witches #2),265,2001-02-06,147
996,997,454,Xenocide (Ender's Saga #3),592,1996-07-15,297
997,998,201,Year of Wonders,358,2002-04-30,212
998,999,94,You Suck (A Love Story #2),328,2007-01-16,331


Таблица books:

book_id — идентификатор книги (int)

author_id — идентификатор автора (int)

title — название книги (varchar)

num_pages — количество страниц (int)

publication_date — дата публикации книги (datetime)

publisher_id — идентификатор издателя (id)

### вывод строк таблицы authors

In [4]:
authors = ''' SELECT * FROM authors'''
print(authors)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd
...,...,...
631,632,William Strunk Jr./E.B. White
632,633,Zadie Smith
633,634,Zilpha Keatley Snyder
634,635,Zora Neale Hurston


Таблица authors

author_id — идентификатор автора (int)

author — имя автора (varchar)

### вывод строк таблицы publishers

In [5]:
publishers = ''' SELECT * FROM publishers'''
print(publishers)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company
...,...,...
335,336,Workman Publishing Company
336,337,Wyatt Book
337,338,Yale University Press
338,339,Yearling


Таблица publishers 

publisher_id — идентификатор издательства (int)

publisher — название издательства (varchar)

### вывод строк таблицы reviews

In [6]:
reviews = ''' SELECT * FROM reviews'''
print(reviews)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...
...,...,...,...,...
2788,2789,999,martinadam,Later hospital turn easy community. Fact same ...
2789,2790,1000,wknight,Change lose answer close pressure. Spend so now.
2790,2791,1000,carolrodriguez,Authority go who television entire hair guy po...
2791,2792,1000,wendy18,Or western offer wonder ask. More hear phone f...


Таблица reviews

review_id — идентификатор обзора (int)

book_id — идентификатор книги (int)

username — имя автора обзора (varchar)

text — текст обзора (varchar)


### вывод строк таблицы ratings

In [7]:
ratings = ''' SELECT * FROM ratings'''
print(ratings)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2
...,...,...,...,...
6451,6452,1000,carolrodriguez,4
6452,6453,1000,wendy18,4
6453,6454,1000,jarvispaul,5
6454,6455,1000,zross,2


Таблица ratings

rating_id — идентификатор оценки (int)

book_id — идентификатор книги (int)

username — имя пользователя, оставившего оценку (varchar)

rating — оценка книги (int)

## проверка на дубликаты

In [8]:
duplicates = ''' 
SELECT title, COUNT(*) AS num_duplicates
FROM books
GROUP BY title
HAVING COUNT(*) > 1;
'''
print(duplicates)


,title,num_duplicates
0,Memoirs of a Geisha,2


In [9]:
duplicates_2 = ''' 
SELECT * FROM books
WHERE title = 'Memoirs of a Geisha';
'''
print(duplicates_2)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,426,39,Memoirs of a Geisha,434,2005-11-15,241
1,427,39,Memoirs of a Geisha,503,2005-11-22,311


Разное кол-во страниц, разные издания - это разные книги, не дубликаты

## задачи 

### Посчитайте, сколько книг вышло после 1 января 2000 года

In [10]:
num_books = '''
SELECT COUNT(*) AS num_books_after_2000
FROM books
WHERE publication_date > '2000-01-01';
'''
print (num_books)

,num_books_after_2000
0,819


После 1 января 2000 года вышло 819 книг

### Для каждой книги посчитайте количество обзоров и среднюю оценку

In [11]:
ct = '''
SELECT 
    b.book_id,
    title,
    COALESCE(r.num_reviews, 0) AS num_reviews,
    COALESCE(ra.avg_rating, 0) AS avg_rating
FROM 
    books b
LEFT JOIN (
    SELECT 
        book_id,
        COUNT(*) AS num_reviews
    FROM 
        reviews
    GROUP BY 
        book_id
) r ON b.book_id = r.book_id
LEFT JOIN (
    SELECT 
        book_id,
        AVG(rating) AS avg_rating
    FROM 
        ratings
    GROUP BY 
        book_id
) ra ON b.book_id = ra.book_id;

'''
print (ct)

,book_id,title,num_reviews,avg_rating
0,652,The Body in the Library (Miss Marple #3),2,4.500000
1,273,Galápagos,2,4.500000
2,51,A Tree Grows in Brooklyn,5,4.250000
3,951,Undaunted Courage: The Pioneering First Missio...,2,4.000000
4,839,The Prophet,4,4.285714
...,...,...,...,...
995,64,Alice in Wonderland,4,4.230769
996,55,A Woman of Substance (Emma Harte Saga #1),2,5.000000
997,148,Christine,3,3.428571
998,790,The Magicians' Guild (Black Magician Trilogy #1),2,3.500000


Сделали таблицу, где содержатся средняя оценка и кол-во обзоров для каждой книги


### Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры

In [12]:
dw = '''
SELECT 
    p.publisher_id,
    p.publisher,
    COUNT(*) AS num_books
FROM 
    books b
JOIN 
    publishers p ON b.publisher_id = p.publisher_id
WHERE 
    b.num_pages > 50
GROUP BY 
    p.publisher_id, p.publisher
ORDER BY 
    COUNT(*) DESC
LIMIT 1;
'''
print (dw)

,publisher_id,publisher,num_books
0,212,Penguin Books,42


Penguin Books выпустило 42 книги (наибольшее среди всех) толще 50-ти страниц

### Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками

In [13]:
rf = '''
WITH popular_books AS (
    SELECT 
        book_id
    FROM 
        ratings
    GROUP BY 
        book_id
    HAVING 
        COUNT(*) >= 50
)
SELECT 
    a.author_id,
    a.author,
    AVG(r.rating) AS avg_rating
FROM 
    authors a
JOIN 
    books b ON a.author_id = b.author_id
JOIN 
    ratings r ON b.book_id = r.book_id
WHERE 
    b.book_id IN (SELECT book_id FROM popular_books)
GROUP BY 
    a.author_id, a.author
ORDER BY 
    AVG(r.rating) DESC
LIMIT 1;
'''
print (rf)

,author_id,author,avg_rating
0,236,J.K. Rowling/Mary GrandPré,4.287097


J.K. Rowling/Mary GrandPré - автор самой высокой средней оценкой 

### Посчитайте среднее количество обзоров от пользователей, которые поставили больше 48 оценок

In [14]:
rs = '''
WITH prolific_users AS (
    SELECT 
        username
    FROM 
        ratings
    GROUP BY 
        username
    HAVING 
        COUNT(*) > 48
)
SELECT 
    AVG(num_reviews) AS avg_reviews
FROM (
    SELECT 
        username,
        COUNT(*) AS num_reviews
    FROM 
        reviews
    WHERE 
        username IN (SELECT username FROM prolific_users)
    GROUP BY 
        username
) AS user_reviews;
'''
print (rs)

,avg_reviews
0,24.0


24 - среднее количество обзоров от пользователей, которые поставили больше 48 оценок 

## вывод

В ходе работы мы получили следующие результаты:

1. После 1 января 2000 года вышло 819 книг.

2. Для каждой книги было посчитано количество обзоров и средняя оценка.

3. Издательство Penguin Books выпустило наибольшее число книг (а именно 42) толще 50 страниц.

4. Автор J.K. Rowling/Mary GrandPré имеет самую высокую среднюю оценку книг с 50 и более оценками.

5. Среднее количество обзоров от пользователей, которые поставили больше 48 оценок, составляет 24.